In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# Load dataset
import pandas as pd
df=pd.read_csv("processed_data - final.csv")

In [ ]:
df.head()

,id,rpm,motor_power,torque,outlet_pressure_bar,air_flow,noise_db,outlet_temp,wpump_outlet_press,water_inlet_temp,...,gaccy,gaccz,haccx,haccy,haccz,bearings,wpump,radiator,exvalve,acmotor
0,1,499,1405.842858,27.511708,1.000,616.579758,40.840517,78.554715,2.960632,43.166392,...,0.383773,2.649801,1.213344,1.409218,2.962484,0,0,0,0,0
1,2,513,1457.370092,31.030115,1.081,615.667472,40.484226,76.902822,2.536711,47.342143,...,0.450954,2.669423,1.210674,1.379050,2.938135,0,0,0,0,0
2,3,495,1582.249959,33.484653,1.369,614.755186,40.918572,77.547021,2.112789,49.306593,...,0.443924,2.772009,1.210612,1.373490,2.991878,0,0,0,0,0
3,4,480,1712.466820,36.394475,1.691,613.950497,40.450953,80.059949,2.087534,46.886933,...,0.370457,2.876056,1.213223,1.443234,3.096158,0,0,0,0,0
4,5,498,1766.035170,38.249154,1.731,613.664264,41.233739,79.130424,2.338877,50.498100,...,0.383868,2.849451,1.209216,1.405190,3.059417,0,0,0,0,0


In [ ]:
df.shape

(1000, 26)

In [ ]:
# Dropping unimportant features
df.drop(['id','oilpump_power','gaccx','gaccy','gaccz','haccx','haccy','haccz','acmotor'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Target variable: air_flow
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
scaled_data = pd.DataFrame(scaled_data, columns=df.columns)
# scaler = StandardScaler()


In [ ]:
X = scaled_data.drop(['outlet_temp'],axis=1)
y = scaled_data['outlet_temp']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# y_train =  y_train.to_numpy().reshape(-1, 1)
# y_test = y_test.to_numpy().reshape(-1, 1)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (700, 16) (700,)
Testing set shape: (300, 16) (300,)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_regressor = RandomForestRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model with best parameters on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mean Squared Error on Test Set:", rmse)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: -0.01825292557699606
Mean Squared Error on Test Set: 0.13772748398495344


In [ ]:
# Building the model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=200, random_state=42,max_depth=10,min_samples_leaf=1,min_samples_split=2)  # You can adjust the number of trees (n_estimators) as needed
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)

In [ ]:
# Predicting on test data
predictions = model.predict(X_test)

In [ ]:
# Calculating RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
rmse=np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 0.13802844750977997
